# Resources

There are a large number of freely available textbooks and courses which cover material in this course. The following resources are recommended; you don't need to read them all, but if you want to follow things up, these are good sources. 

## Machine Learning / Statistics

[Introduction to Statistical Learning](http://www.r-bloggers.com/in-depth-introduction-to-machine-learning-in-15-hours-of-expert-videos/) A thorough introduction to statistical learning, with both a textbook and an accompanying video lecture series. Uses R for exercises. The classic textbook ["The elements of statistical learning"](http://statweb.stanford.edu/~tibs/ElemStatLearn/) is also available for free online.

[Deep Learning](http://www-labs.iro.umontreal.ca/~bengioy/DLbook/) This freely-available textbook goes into great depth on deep learning, but the introductory chapters in Part 1 are an *excellent* introduction to the concepts needed understand machine learning topics.

[Information Theory, Inference and Learning Algorithms](http://www.inference.phy.cam.ac.uk/itila/) A dense and insightful exploration of machine learning and information theory; requires serious study but explains the mathematical underpinnings of machine learning clearly and succinctly.

[Machine Learning: A Probabilistic Perspective](http://www.cs.ubc.ca/~murphyk/MLbook/) Probably the best all-round machine learning textbook.  Covers a vast swathe of material in a fairly accessible manner. **Not freely available**

[Machine Learning](https://www.coursera.org/course/ml) Andrew Ng's excellent short course on machine learning, available as a Coursera on-demand video lecture series.

[Probability and Statistics Cookbook](http://matthias.vallentin.net/probability-and-statistics-cookbook/) If you need a formula in probability or statistics, it's probably in here. A very compact reference book.

## Control theory

[Control Theory for Humans](http://www.amazon.co.uk/Control-Theory-Humans-Quantitative-Performance/dp/0805822933) A high-level but accessible introduction to control theory. **Not freely available**

[Manual Control -- theory and applications](http://www.dtic.mil/cgi-bin/GetTRDoc?AD=AD0449586) **PDF** An old (1964!) but very thorough treatment of manual control (i.e. human operator performance).



## Optimisation

[Convex Optimization](hhttps://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf) **PDF** A very mathematical but complete coverage of convex optimisation.

